In [2]:
# 1. Create a Flask app that displays "Hello, World!" on the homepage.
pip install flask
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 2. Build a Flask app with static HTML pages and navigate between them.
!pip install flask
from flask import Flask, render_template_string

app = Flask(__name__)

# Templates for two pages
home_page = """<h1>Home Page</h1><a href="/about">Go to About Page</a>"""
about_page = """<h1>About Page</h1><a href="/">Go to Home Page</a>"""

@app.route('/')
def home():
    return render_template_string(home_page)

@app.route('/about')
def about():
    return render_template_string(about_page)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



In [ ]:
# 3. Develop a Flask app that uses URL parameters to display dynamic content.
!pip install flask
from flask import Flask

app = Flask(__name__)

@app.route('/greet/<name>')
def greet(name):
    return f"Hello, {name}! Welcome to the dynamic Flask app."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 4. Create a Flask app with a form that accepts user input and displays it.
!pip install flask
from flask import Flask, request, render_template_string

app = Flask(__name__)

form_page = """
    <form method="post" action="/submit">
        Enter your name: <input type="text" name="name">
        <input type="submit" value="Submit">
    </form>
"""

@app.route('/')
def form():
    return render_template_string(form_page)

@app.route('/submit', methods=['POST'])
def submit():
    name = request.form.get('name')
    return f"Hello, {name}! Your form has been submitted."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 5. Implement user sessions in a Flask app to store and display user-specific data.
!pip install flask
from flask import Flask, session, request, redirect, url_for

app = Flask(__name__)
app.secret_key = 'supersecretkey'

@app.route('/')
def home():
    if 'username' in session:
        return f"Welcome back, {session['username']}!"
    return "You are not logged in. <a href='/login'>Login here</a>"

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('home'))
    return """
        <form method="post">
            Enter Username: <input type="text" name="username">
            <input type="submit" value="Login">
        </form>
    """

@app.route('/logout')
def logout():
    session.pop('username', None)
    return "You have been logged out."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 6. Build a Flask app that allows users to upload files and display them on the website.
!pip install flask
from flask import Flask, request, render_template_string
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '/content/uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

upload_page = """
    <h1>File Upload</h1>
    <form method="post" action="/upload" enctype="multipart/form-data">
        <input type="file" name="file">
        <input type="submit" value="Upload">
    </form>
"""

@app.route('/')
def home():
    return render_template_string(upload_page)

@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files['file']
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return f"File uploaded successfully: <a href='/view/{filename}'>{filename}</a>"

@app.route('/view/<filename>')
def view_file(filename):
    return f"<h1>Uploaded File: {filename}</h1><img src='/content/uploads/{filename}' width='300'>"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.
!pip install flask
import sqlite3
from flask import Flask, request, jsonify

app = Flask(__name__)
DATABASE = '/content/items.db'

# Database setup
def init_db():
    with sqlite3.connect(DATABASE) as conn:
        conn.execute("CREATE TABLE IF NOT EXISTS items (id INTEGER PRIMARY KEY, name TEXT)")
        conn.commit()

init_db()

@app.route('/items', methods=['GET'])
def get_items():
    with sqlite3.connect(DATABASE) as conn:
        items = conn.execute("SELECT * FROM items").fetchall()
    return jsonify(items)

@app.route('/items', methods=['POST'])
def add_item():
    name = request.json['name']
    with sqlite3.connect(DATABASE) as conn:
        conn.execute("INSERT INTO items (name) VALUES (?)", (name,))
        conn.commit()
    return jsonify({"message": "Item added"}), 201

@app.route('/items/<int:item_id>', methods=['PUT'])
def update_item(item_id):
    name = request.json['name']
    with sqlite3.connect(DATABASE) as conn:
        conn.execute("UPDATE items SET name = ? WHERE id = ?", (name, item_id))
        conn.commit()
    return jsonify({"message": "Item updated"})

@app.route('/items/<int:item_id>', methods=['DELETE'])
def delete_item(item_id):
    with sqlite3.connect(DATABASE) as conn:
        conn.execute("DELETE FROM items WHERE id = ?", (item_id,))
        conn.commit()
    return jsonify({"message": "Item deleted"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 8. Implement user authentication and registration in a Flask app using Flask-Login.
!pip install flask flask-login
from flask import Flask, request, redirect, url_for, render_template_string
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.secret_key = 'supersecretkey'
login_manager = LoginManager()
login_manager.init_app(app)

# User setup
class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

users = {'testuser': User(1, 'testuser', 'password')}

@login_manager.user_loader
def load_user(user_id):
    for user in users.values():
        if user.id == int(user_id):
            return user
    return None

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = users.get(username)
        if user and user.password == password:
            login_user(user)
            return redirect(url_for('dashboard'))
    return """
        <form method="post">
            Username: <input type="text" name="username">
            Password: <input type="password" name="password">
            <input type="submit" value="Login">
        </form>
    """

@app.route('/dashboard')
@login_required
def dashboard():
    return f"Hello, {current_user.username}! Welcome to your dashboard."

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return "You have been logged out."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.
!pip install flask
from flask import Flask, jsonify, request

app = Flask(__name__)

books = []

@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)

@app.route('/books', methods=['POST'])
def add_book():
    book = request.json
    books.append(book)
    return jsonify({"message": "Book added"}), 201

@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book_data = request.json
    for book in books:
        if book['id'] == book_id:
            book.update(book_data)
            return jsonify({"message": "Book updated"})
    return jsonify({"message": "Book not found"}), 404

@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({"message": "Book deleted"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 10. Design a Flask app with proper error handling for 404 and 500 errors.
!pip install flask
from flask import Flask, render_template_string

app = Flask(__name__)

@app.route('/')
def home():
    return "<h1>Welcome to the Home Page</h1>"

@app.errorhandler(404)
def not_found(e):
    return render_template_string("<h1>404 Error</h1><p>Page not found.</p>"), 404

@app.errorhandler(500)
def server_error(e):
    return render_template_string("<h1>500 Error</h1><p>Internal server error.</p>"), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# 11. Create a real-time chat application using Flask-SocketIO.
!pip install flask flask-socketio
from flask import Flask, render_template_string, request
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

chat_template = """
    <!doctype html>
    <html>
        <head><title>Chat</title></head>
        <body>
            <h2>Real-Time Chat</h2>
            <ul id="messages"></ul>
            <input type="text" id="message" autocomplete="off" placeholder="Type your message here"/>
            <button onclick="sendMessage()">Send</button>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.min.js"></script>
            <script>
                var socket = io();
                socket.on('message', function(msg){
                    var li = document.createElement("li");
                    li.appendChild(document.createTextNode(msg));
                    document.getElementById("messages").appendChild(li);
                });
                function sendMessage() {
                    var msg = document.getElementById("message").value;
                    socket.send(msg);
                    document.getElementById("message").value = '';
                }
            </script>
        </body>
    </html>
"""

@app.route('/')
def index():
    return render_template_string(chat_template)

@socketio.on('message')
def handle_message(msg):
    print(f"Message: {msg}")
    send(msg, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000)


In [ ]:
# 12. Build a Flask app that updates data in real-time using WebSocket connections.
!pip install flask flask-socketio
from flask import Flask, render_template_string
from flask_socketio import SocketIO
import random, time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

template = """
    <!doctype html>
    <html>
        <head><title>Real-Time Data Update</title></head>
        <body>
            <h2>Real-Time Data</h2>
            <p>Random Number: <span id="number">0</span></p>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.min.js"></script>
            <script>
                var socket = io();
                socket.on('update_number', function(data){
                    document.getElementById("number").textContent = data.number;
                });
            </script>
        </body>
    </html>
"""

@app.route('/')
def index():
    return render_template_string(template)

def update_data():
    while True:
        number = random.randint(1, 100)
        socketio.emit('update_number', {'number': number})
        time.sleep(2)

@socketio.on('connect')
def handle_connect():
    socketio.start_background_task(target=update_data)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000)


In [ ]:
# 13. Implement notifications in a Flask app using WebSockets to notify users of updates.
!pip install flask flask-socketio
from flask import Flask, render_template_string
from flask_socketio import SocketIO
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

notification_template = """
    <!doctype html>
    <html>
        <head><title>Notifications</title></head>
        <body>
            <h2>User Notifications</h2>
            <ul id="notifications"></ul>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.min.js"></script>
            <script>
                var socket = io();
                socket.on('notification', function(msg){
                    var li = document.createElement("li");
                    li.appendChild(document.createTextNode(msg));
                    document.getElementById("notifications").appendChild(li);
                });
            </script>
        </body>
    </html>
"""

@app.route('/')
def index():
    return render_template_string(notification_template)

def send_notifications():
    messages = ["Welcome to the site!", "Check out our latest updates!", "Don't miss our special offers!"]
    while True:
        for message in messages:
            socketio.emit('notification', message)
            time.sleep(5)

@socketio.on('connect')
def handle_connect():
    socketio.start_background_task(target=send_notifications)

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5000)
